# split

> Fill in a module description here

In [ ]:
#| default_exp split

In [2]:
#export
import subprocess
from fastai.vision.all import *
from tqdm import tqdm
import cv2
import argparse

In [ ]:
#export
def extract_frames_from_video(video_path, output_dir, unique_capture_name, interval=1):
    """
    Extracts frames from a given video using ffmpeg at a specified interval and saves them to the output directory.
    If interval is set to -1, all frames from the video are extracted.

    Args:
        video_path (Path): Path to the input video file.
        output_dir (Path): Directory to save the extracted frames.
        unique_capture_name (str): Name of the unique capture folder to include in the frame filenames.
        interval (int): Interval in seconds to extract frames. If -1, extract all frames.

    Returns:
        None
    """
    video_name = video_path.stem
    output_dir.mkdir(parents=True, exist_ok=True)

    # Define the ffmpeg command to extract frames
    frame_output_pattern = f"{output_dir}/{unique_capture_name}_{video_name}_%04d.jpg"
    command = [
        'ffmpeg',
        '-i', str(video_path),
        '-vf', f'fps=1/{interval}' if interval != -1 else '',
        '-q:v', '2',
        frame_output_pattern
    ]

    # Run the ffmpeg command using subprocess with suppressed output
    subprocess.run([arg for arg in command if arg], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

In [ ]:
#export
def process_dataset_videos(dataset_root_path='dataset', interval=1):
    """
    Process all videos in the dataset directory and extract frames to a frames folder using a single progress bar.

    Args:
        dataset_root_path (str): Path to the main dataset directory. Default is 'dataset'.
        interval (int): Interval in seconds to extract frames from videos. If -1, extract all frames.

    Returns:
        None
    """
    dataset_root_path = Path(dataset_root_path)
    videos_dir = dataset_root_path / 'videos'
    frames_dir = dataset_root_path / 'frames'

    video_files = get_files(videos_dir, extensions='.mp4')

    with tqdm(total=len(video_files), desc="Extracting frames from all videos", unit='video') as pbar:
        for capture_folder in videos_dir.ls():
            if capture_folder.is_dir(): 
                print(f"Processing folder: {capture_folder.name}")
                video_files_in_folder = get_files(capture_folder, extensions='.mp4')

                for video_path in video_files_in_folder:
                    extract_frames_from_video(video_path, frames_dir, capture_folder.name, interval)
                    pbar.update(1)

In [ ]:
#export
def main():
    """
    Main function to handle command-line arguments and execute the frame extraction process.
    """
    parser = argparse.ArgumentParser(description="Extract frames from videos in the dataset directory.")
    parser.add_argument('--dataset_root_path', type=str, default='../temset', help='Path to the dataset directory containing videos.')
    parser.add_argument('--interval', type=int, default=1, help='Interval in seconds for frame extraction. Use -1 to extract all frames.')

    args = parser.parse_args()
    process_dataset_videos(args.dataset_root_path, args.interval)

if __name__ == "__main__":
    main()